In [1]:
%matplotlib inline

In [2]:
import fipy as fp

In [3]:
from fipy import numerix as nmx

In [5]:
N = 200
mesh = fp.Grid3D(nx=N, ny=N, nz=1)

In [6]:
C = fp.CellVariable(mesh=mesh, name="C", hasOld=True)

In [7]:
Calpha = 0.05
Cbeta = 0.95
Cm = (Calpha + Cbeta) / 2.
A = 2.0
B = A / (Calpha - Cm)**2
D = Dalpha = Dbeta = 2 / (Cbeta - Calpha)
kappa = 2.0

In [8]:
f0 = -(A/2)*(C - Cm)**2 + (B/4)*(C - Cm)**4 + (Calpha/4)*(C - Calpha)**4 + (Cbeta/4)*(C - Cbeta)**4

In [9]:
f = f0 + (kappa/2.)*(C.grad.mag)**2
f.name = "f"

In [10]:
Cf = C.arithmeticFaceValue
eq = (fp.TransientTerm(coeff=1.)
      == fp.DiffusionTerm(coeff=D*(-A + 3*B*(Cf - Cm)**2 + 3*Calpha*(Cf - Calpha)**2 + 3*Cbeta*(Cf - Cbeta)**2))
      - fp.DiffusionTerm(coeff=(D,kappa)))

In [12]:
epsilon = 0.01
q = [[nmx.sqrt(2.)], [nmx.sqrt(3.)], [1.]]
C.value = 0.45 + epsilon * nmx.cos(nmx.dot(q, mesh.cellCenters))

In [13]:
dt = 0.01
elapsed = 0.
saved = elapsed
duration = 10000.
step = 0

In [16]:
import os

In [18]:
direc = "1b-3d"

In [19]:
os.mkdir(direc)

In [20]:
fp.tools.dump.write(mesh, filename=os.path.join(direc, "mesh.dmp"))

In [22]:
fp.TSVViewer(vars=C).plot(filename=os.path.join(direc, "{step}.gz".format(step=0)))

In [23]:
with open(os.path.join(direc, "stats.txt"), 'w') as stats:
    stats.write("\t".join(["step", "t", "dt", "Cmin", "Cmax", "f"]) + "\n")

In [24]:
while elapsed < duration:
    C.updateOld()
    for sweep in range(6):
        res = eq.sweep(C, dt=dt) #, solver=fp.LinearLUSolver())
    if res < 0.1:
        step += 1
        elapsed += dt
        dt *= 1.1
        if elapsed - saved > 10:
            fp.TSVViewer(vars=C).plot(filename=os.path.join(direc, "{step}.gz".format(step=step)))
            # viewer.plot()
            saved = elapsed
        with open(os.path.join(direc, "stats.txt"), 'a') as stats:
            stats.write("\t".join([str(it) for it in [step, elapsed, dt, min(C), max(C), 
                                                      f.cellVolumeAverage * mesh.cellVolumes.sum()]]) + "\n")
    else:
        dt *= 0.8
        C.value = C.old.value


KeyboardInterrupt: 